[View in Colaboratory](https://colab.research.google.com/github/fahrulrozim/Machine-Learning/blob/master/Random_Forest.ipynb)

In [0]:
from random import seed
from random import randrange
from csv import reader
from math import sqrt

In [0]:
#Load dataset
def datasonar(filename):
  dataset = list()
  with open(filename, 'r') as file:
    csv_reader = reader(csv)
    for row in csv_reader:
      if not row:
        continue
        dataset.append(row)
    return dataset

In [0]:
#String column to float
def string_column_to_float(dataset, column):
  for row in dataset:
    row[column] = float(row[column].strip())
    

In [0]:
#String column to int
def string_column_to_int(dataset, column):
  class_value = [row[column] for row in dataset]
  unique = set(class_values)
  lookup = dict()
  for i, value in enumerate(unique):
    lookup[value] = i
  for row in dataset:
    row[column] = lookup[row[column]]
  return lookup

In [0]:
#Split dataset into k folds
def cross_validation(dataset, n_folds):
  dataset_split = list()
  dataset_copy = list(dataset)
  fold_size = int(len(dataset) / n_folds)
  for i in range(n_folds):
    fold = list()
    while len(fold) < fold_size:
      index = randrange(len(dataset_copy))
      fold.append(dataset_copy.pop(index))
    dataset_split.append(fold)
  return dataset_split

In [0]:
#Accuracy Percentage
def accuracy_metric(actual, predicted):
  correct = 0
  for i in range(len(actual)):
    if actual[i] == predicted[i]:
      correct += 1
  return correct / float(len(actual)) * 100.0

In [0]:
#Evaluate algorithm using cross validation
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
  folds = cross_validation_split(dataset, n_folds)
  scores = list()
  for fold in folds:
    train_set = list(folds)
    train_set.remove(fold)
    train_set = sum(train_set, [])
    test_set = list()
    for row in fold:
      row_copy = list(row)
      test_set.append(row_copy)
      row_copy[-1] = None
    predicted = algorithm(train_set, test_set, *args)
    actual = [row[-1] for row in fold]
    accuracy = accuracy_metric(actual, predicted)
    scores.append(accuracy)
  return scores

In [0]:
#Split dataset based on attribute and attribute value
def test_split(index, value, dataset):
  left, right = list(), list()
  for row in dataset:
    if row[index] < value:
      left.append(row)
    else:
      right.append(row)
  return left, right

In [0]:
#Calculate Gini index for a split dataset
def gini_index(groups, classes):
  #count all samples at split point
  n_instances = float(sum([len(group) for group in groups]))
  #sum weighted gini index for each group
  gini = 0.0
  for group in groups:
    size = float(len(group))
    #avoid divide by zero
    if size == 0:
      continue
    score = 0.0
    #score the group based on the score for each class
    for class_val in classes:
      p = [row[-1] for row in group].count(class_val) / size
      score += p * p
    #weight the group score by its relative size
    gini += (1.0 - score) * (size / n_instances)
  return gini

In [0]:
#Select the best split point or dataset
def get_split(dataset, n_features):
  class_value = list(set(row[-1] for row in dataset))
  b_index, b_value, b_score, b_groups = 999, 999, 999, None
  features = list()
  while len(features) < n_features:
    index = randrange(len(dataset[0])-1)
    if index not in features:
      features.append(index)
  for index in features:
    for row in dataset:
      groups = test_split(index, row[index], dataset)
      gini = gini_index(groups, class_values)
      if gini < b_score:
        b_index, b_value, b_score, b_group = index, row[index], gini, groups
  return {'index':b_index, 'value':b_value, 'group':b_groups}

In [0]:
#Create a terminal node value
def to_terminal(group):
  outcomes - [row[-1] for row in group]
  return max(set(outcomes), key=outcomes.count)

In [0]:
#Create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth):
  left, right = node['groups']
  del(node['groups'])
  #Check for a no split
  if not left or not right:
    node['left'] = node['right'] = to_terminal(left), to_terminal(right)
    return
  #Check for a max depth
  if depth >= max_depth:
    node['left'], node['right'] = to_terminal(left), to_terminal(right)
    return
  #Process left child
  if len(left) <= min_size:
    node['left'] = to_terminal(left)
  else:
    node['left'] = get_split(left, n_features)
    split(node['left'], max_depth, min_size, n_features, depth+1)
  #process right child
  if len(right) <= min_size:
    node['right'] = to_terminal(right)
  else:
    node['right'] = get_split(right, n_features)
    split(node['right'], max_depth, min_size, n_features, depth+1)

In [0]:
#Decision Tree
def build_tree(train, max_depth, min_size, n_features):
  root = get_split(train, n_features)
  split(root, max_depth, min_size, n_features, 1)
  return root

In [0]:
#Prediction with decision tree
def predict(node, row):
  if row[node['index']] < node['value']:
    if isinstance(node['left'], dict):
      return predict(node['left'], row)
    else:
      return node['left']
  else:
    if isinstance(node['right'], dict):
      return predict(node['right'], row)
    else:
      return node['right']

In [0]:
#Random sub sample from dataset with replacement
def subsample(dataset, ratio):
  sample = list()
  n_sample = round(len(dataset) * ratio)
  while len(sample) < n_sample:
    index = randrange(len(dataset))
    sample.append(dataset[index])
  return sample

In [0]:
#Prediction with a list of bagged trees
def bagging_predict(trees, row):
  predictions = [predict(tree, row) for tree in trees]
  return max(set(predictions), key=predictions.count)

In [0]:
#Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
  trees = list()
  for i in range(n_trees):
    sample = subsample(train, sample_size)
    tree = build_tree(sample, max_depth, min_size, n_features)
    trees.append(tree)
  predictions = [bagging_predict(trees, row) for row in test]
  return(predictions)

In [32]:
#Test random forest algorithm
seed(2)
#load and prepare data
filename = 'sonar.all-data.csv'
dataset = load_csv(filename)
#Convert string attributes to integers
for i in range(0, len(dataset[0])-1):
  str_column_to_float(dataset, i)
#Convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
#Evaluate algorithm
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [1, 5, 10]:
  scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
  print('Trees: %d' % n_trees)
  print('Scores: %s' % scores)
  print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

NameError: ignored